In [2]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import mediapipe as mp
import time

In [3]:
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

In [4]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.multi_hand_landmarks)

def draw_styled_landmarks(image, results):
    if results.multi_hand_landmarks != None:
        mp_drawing.draw_landmarks(image, results.multi_hand_landmarks[0], mp_hands.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 ) 

def extract_keypoints(results):
    keypoints = np.zeros(21 * 3)

    if results.multi_hand_landmarks:
        hand_landmarks = results.multi_hand_landmarks[0]
        keypoints = np.array([[landmark.x, landmark.y, landmark.z] for landmark in hand_landmarks.landmark]).flatten()
    
    return keypoints

In [ ]:
import os

#Add abs path to desired folder
DATA_PATH = os.path.abspath("")
print(DATA_PATH)
actions = np.array(['hola', 'j', 'no'])

no_sequences = 30
sequence_length = 30
start_folder = 30

In [14]:
for action in actions: 
    action_path = os.path.join(DATA_PATH, action)
    
    # Create action folder if it doesn't exist
    if not os.path.exists(action_path):
        os.makedirs(action_path)
    
    # Create sequence folders
    for sequence in range(0, no_sequences):
        try: 
            os.makedirs(os.path.join(action_path, str(sequence)))
        except FileExistsError:
            print(f"Sequence folder '{sequence}' already exists for action '{action}'.")
        except Exception as e:
            print(f"Error creating sequence folder for action '{action}': {e}")

In [ ]:
cap = cv2.VideoCapture(0)

# Set mediapipe model 
with mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5) as hands:
    
    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        for sequence in range(0, 30):
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, hands)

                # Draw landmarks
                draw_styled_landmarks(image, results)
                
                # Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(500)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                
                # Export keypoints
                keypoints = extract_keypoints(results)
                
                # Create directories if they don't exist
                npy_dir = os.path.join(DATA_PATH, action, str(sequence))
                os.makedirs(npy_dir, exist_ok=True)
                
                # Save keypoints
                npy_path = os.path.join(npy_dir, str(frame_num) + '.npy')
                np.save(npy_path, keypoints)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break

cap.release()
cv2.destroyAllWindows()